In [1]:
!mkdir -p app
!mkdir -p tests


In [2]:
!pip install fastapi uvicorn python-dotenv requests


In [3]:
%%writefile requirements.txt
fastapi==0.100.0
uvicorn==0.22.0
python-dotenv==1.0.0
requests==2.31.0


Writing requirements.txt


In [4]:
%%writefile app/__init__.py
__all__ = ["main", "agent", "tools", "memory"]


Writing app/__init__.py


In [5]:
%%writefile app/tools.py
from typing import Optional

# Simple knowledge base
KB = {
    "python": "Python is a high-level, interpreted programming language.",
    "fastapi": "FastAPI is a fast Python framework for building APIs.",
    "uvicorn": "Uvicorn is a lightning-fast ASGI server for Python.",
    "ai": "AI (Artificial Intelligence) is the simulation of human intelligence by machines."
}

def search_tool(query: str) -> Optional[str]:
    """Very simple keyword-based search tool."""
    q = query.lower().strip()

    if q in KB:
        return KB[q]

    for key, value in KB.items():
        if key in q:
            return value

    return None


Writing app/tools.py


In [6]:
%%writefile app/memory.py
from collections import deque
from typing import List

class ShortTermMemory:
    """Stores the last few messages to maintain context."""
    def __init__(self, capacity: int = 3):
        self.capacity = capacity
        self.messages = deque(maxlen=capacity)

    def add(self, message: str):
        self.messages.append(message)

    def get_context(self) -> List[str]:
        return list(self.messages)

    def clear(self):
        self.messages.clear()


Writing app/memory.py


In [7]:
%%writefile app/agent.py
from typing import Dict, Any
from .tools import search_tool
from .memory import ShortTermMemory

memory = ShortTermMemory(capacity=3)

def is_factual_question(message: str) -> bool:
    """Simple detection of factual questions using keywords."""
    message = message.lower()
    question_words = ["what", "when", "who", "where", "which", "explain", "define"]

    if any(message.startswith(w) for w in question_words):
        return True

    if "about" in message:
        return True

    return False

def generate_conversational_reply(message: str, context: list) -> str:
    """Very simple conversational response."""
    if "hi" in message.lower() or "hello" in message.lower():
        return "Hey! How can I help you today?"
    return "Got it! You can ask me a factual question like 'What is Python?'."

def agent_respond(message: str) -> Dict[str, Any]:
    memory.add(f"user: {message}")

    factual = is_factual_question(message)

    response = {
        "input": message,
        "factual": factual,
        "tool_used": False,
        "answer": None,
        "context": memory.get_context()
    }

    if factual:
        tool_answer = search_tool(message)
        if tool_answer:
            response["tool_used"] = True
            response["answer"] = tool_answer
            memory.add(f"agent: {tool_answer}")
            return response

    reply = generate_conversational_reply(message, memory.get_context())
    response["answer"] = reply
    memory.add(f"agent: {reply}")
    return response

def reset_memory():
    memory.clear()


Writing app/agent.py


In [8]:
%%writefile app/main.py
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from .agent import agent_respond, reset_memory

app = FastAPI(title="AI Question-Answer Helper")

class ChatRequest(BaseModel):
    message: str

class ChatResponse(BaseModel):
    input: str
    factual: bool
    tool_used: bool
    answer: str
    context: list

@app.post("/chat", response_model=ChatResponse)
async def chat(req: ChatRequest):
    if not req.message.strip():
        raise HTTPException(status_code=400, detail="Message cannot be empty")

    reply = agent_respond(req.message)
    return ChatResponse(**reply)

@app.post("/reset-memory")
async def reset():
    reset_memory()
    return {"status": "ok", "message": "Memory cleared"}

# Run with:
# uvicorn app.main:app --reload --port=8000


Writing app/main.py


In [10]:
!pip install fastapi uvicorn nest_asyncio pyngrok


In [23]:
import nest_asyncio
import uvicorn
from fastapi import FastAPI
from threading import Thread

nest_asyncio.apply()

app = FastAPI()

@app.get("/")
def home():
    return {"message": "API running!"}

def run():
    uvicorn.run(app, host="0.0.0.0", port=8000)

Thread(target=run).start()


INFO:     Started server process [164]
INFO:     Waiting for application startup.
INFO:     Application startup complete.


In [24]:
import requests

response = requests.get("http://localhost:8000/")
response.json()


INFO:     127.0.0.1:37696 - "GET / HTTP/1.1" 200 OK


{'message': 'API running!'}

In [26]:
from app.agent import agent_respond

print(agent_respond("who is the president of nigeria"))
print(agent_respond("tell me something about football"))
print(agent_respond("how many states are in nigeria"))
print(agent_respond("ok cool, continue"))

{'input': 'who is the president of nigeria', 'factual': True, 'tool_used': False, 'answer': "Got it! You can ask me a factual question like 'What is Python?'.", 'context': ['user: ok cool, continue', "agent: Got it! You can ask me a factual question like 'What is Python?'.", 'user: who is the president of nigeria']}
{'input': 'tell me something about football', 'factual': True, 'tool_used': False, 'answer': 'Hey! How can I help you today?', 'context': ['user: who is the president of nigeria', "agent: Got it! You can ask me a factual question like 'What is Python?'.", 'user: tell me something about football']}
{'input': 'how many states are in nigeria', 'factual': False, 'tool_used': False, 'answer': "Got it! You can ask me a factual question like 'What is Python?'.", 'context': ['user: tell me something about football', 'agent: Hey! How can I help you today?', 'user: how many states are in nigeria']}
{'input': 'ok cool, continue', 'factual': False, 'tool_used': False, 'answer': "Got it

In [27]:
import nest_asyncio
import uvicorn
from fastapi import FastAPI
from threading import Thread

nest_asyncio.apply()

app = FastAPI()

from app.agent import agent_respond
from pydantic import BaseModel

class ChatRequest(BaseModel):
    message: str

@app.post("/chat")
async def chat(req: ChatRequest):
    return agent_respond(req.message)

def run():
    uvicorn.run(app, host="0.0.0.0", port=8000)

Thread(target=run).start()


INFO:     Started server process [164]


In [30]:
import requests

response = requests.post(
    "http://127.0.0.1:8000/chat",
    json={"message": "who is the president of nigeria"}
)
response.json()


INFO:     127.0.0.1:50570 - "POST /chat HTTP/1.1" 404 Not Found


{'detail': 'Not Found'}

In [32]:
# Install necessary packages
!pip install fastapi uvicorn nest_asyncio requests

# ----------------------------
# Step 1: Define agent functions
# ----------------------------
memory = []

def add_to_memory(user_msg, assistant_msg):
    memory.append({"user": user_msg, "assistant": assistant_msg})
    if len(memory) > 5:
        memory.pop(0)

# Small knowledge base
knowledge_base = {
    "python": "Python is a high-level, interpreted programming language.",
    "fastapi": "FastAPI is a fast Python framework for building APIs.",
    "uvicorn": "Uvicorn is a lightning-fast ASGI server for Python.",
    "ai": "AI (Artificial Intelligence) is the simulation of human intelligence by machines.",
}

def search_tool(query):
    q = query.lower().strip()
    return knowledge_base.get(q, None)

def agent_respond(message):
    # Check factual
    tool_answer = search_tool(message)
    if tool_answer:
        add_to_memory(message, tool_answer)
        return {
            "input": message,
            "factual": True,
            "tool_used": True,
            "answer": tool_answer,
            "context": memory
        }

    # Fallback conversational
    reply = f"Got it! You can ask me a factual question like 'What is Python?'."
    add_to_memory(message, reply)
    return {
        "input": message,
        "factual": False,
        "tool_used": False,
        "answer": reply,
        "context": memory
    }

# ----------------------------
# Step 2: Start FastAPI
# ----------------------------
import nest_asyncio
import uvicorn
from fastapi import FastAPI
from pydantic import BaseModel
from threading import Thread

nest_asyncio.apply()

app = FastAPI(title="AI Question-Answer Helper")

class ChatRequest(BaseModel):
    message: str

@app.on_event("startup")
async def startup_event():
    print(" FastAPI server has started successfully!")

@app.post("/chat")
async def chat(req: ChatRequest):
    return agent_respond(req.message)

def run():
    # Changed port to 8001 to avoid conflicts
    uvicorn.run(app, host="0.0.0.0", port=8001)

# Start FastAPI in background thread
Thread(target=run).start()

# ----------------------------
# Step 3: Test multiple queries
# ----------------------------
import time
import requests

# Wait a few seconds for the server to start
time.sleep(3)

test_queries = [
    "What is Python?",
    "Tell me about AI",
    "Who is the president of Nigeria?",
    "Hello there!",
    "What is FastAPI?"
]

print("===== Testing /chat endpoint =====\n")
for query in test_queries:
    # Updated port in the request URL
    response = requests.post(
        "http://127.0.0.1:8001/chat",
        json={"message": query}
    )
    print(f"User: {query}")
    # Added error handling for responses that don't contain 'answer'
    try:
        print(f"Agent: {response.json()['answer']}\n")
    except KeyError:
        print(f"Agent: Error - Status Code: {response.status_code}, Detail: {response.json().get('detail', 'No detail provided')}\n")
    except Exception as e:
        print(f"Agent: An unexpected error occurred: {e}\n")


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-41' coro=<Server.serve() done, defined at /usr/local/lib/python3.12/dist-packages/uvicorn/server.py:69> exception=SystemExit(1)>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/server.py", line 164, in startup
    server = await loop.create_server(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/asyncio/base_events.py", line 1584, in create_server
    raise OSError(err.errno, msg) from None
OSError: [Errno 98] error while attempting to bind on address ('0.0.0.0', 8000): [errno 98] address already in use

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.12/threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipython-input-3657850984.py", li

/tmp/ipython-input-2612402754.py:66: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("startup")
INFO:     Started server process [164]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8001 (Press CTRL+C to quit)


 FastAPI server has started successfully!
===== Testing /chat endpoint =====

INFO:     127.0.0.1:50200 - "POST /chat HTTP/1.1" 200 OK
User: What is Python?
Agent: Got it! You can ask me a factual question like 'What is Python?'.

INFO:     127.0.0.1:50210 - "POST /chat HTTP/1.1" 200 OK
User: Tell me about AI
Agent: Got it! You can ask me a factual question like 'What is Python?'.

INFO:     127.0.0.1:50212 - "POST /chat HTTP/1.1" 200 OK
User: Who is the president of Nigeria?
Agent: Got it! You can ask me a factual question like 'What is Python?'.

INFO:     127.0.0.1:50214 - "POST /chat HTTP/1.1" 200 OK
User: Hello there!
Agent: Got it! You can ask me a factual question like 'What is Python?'.

INFO:     127.0.0.1:50216 - "POST /chat HTTP/1.1" 200 OK
User: What is FastAPI?
Agent: Got it! You can ask me a factual question like 'What is Python?'.

